In [1]:
import torch
import torch.nn as nn
import numpy as np

from einops import rearrange

from mosaic_sdf import MosaicSDF
from shape_sampler import ShapeSampler
from optimizer import MosaicSDFOptimizer
from mosaic_sdf_visualizer import MosaicSDFVisualizer

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

In [2]:
cube_mesh_path = 'data/cube.obj'
teapot_mesh_path = 'data/utah_teapot.obj'
cow_mesh_path = 'data/cow_mesh/cow.obj'

cube_wireframe_path = 'data/cube_wireframe.obj'

sdf_shape_path = teapot_mesh_path

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
shape_sampler = ShapeSampler.from_file(sdf_shape_path, device='cuda')

config = {   
    'device': device,
    # 'shape_sampler': shape_sampler,  # Adjust accordingly
    'shape_path': sdf_shape_path,  # Adjust accordingly
    
    # mosaicSDF params
    'grid_resolution': 7,
    # 'n_grids': 1024,
    'n_grids': 128,
    # 'n_grids': 8,
    'points_random_spread': .03,
    'mosaic_scale_multiplier': 3,
    
    # optimizer params
    # TODO play with Adam params
    'lr': 1e-4,
    'weight_decay': 0.0,

    'lambda_val': .1,
    
    
    # optimization params
    'points_in_epoch': 2048,
    'points_sample_size': 32,
    'gradient_accumulation_steps': 1,

    'eval_every_nth_points': 256,
    'val_size': 512,
    'points_sample_size_eval_scaler': 4, # can sample faster during eval

    'project_name': 'mosaicSDF_smoke',
    'log_to_wandb': False, 

    # other debug stuff
    'output_graph': False,
    'points_random_sampling': False
}

optimizer = MosaicSDFOptimizer(config)

optimizer.model.update_sdf_values(shape_sampler)


/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/point_cloud_utils/_mesh_io.py:338: RuntimeWarning: TinyObjReader: Empty group name. line: 6
Empty group name. line: 6475

  mesh_dict = load_mesh_internal(mesh_filename, dtype)
/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/ray/tune/trainable/trainable.py:695: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  self._result_logger = UnifiedLogger(config, self._logdir, loggers=None)
/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonL

In [5]:
optimizer.step()


Iteration 7, val Loss: 0.0087, val L1: 0.0057, val L2: 0.0361 ||| train Loss: 0.0129 train L1: 0.0055, train L2: 0.0804 
Iteration 15, val Loss: 0.0088, val L1: 0.0057, val L2: 0.0362 ||| train Loss: 0.0124 train L1: 0.0061, train L2: 0.0690 
Iteration 23, val Loss: 0.0088, val L1: 0.0057, val L2: 0.0362 ||| train Loss: 0.0130 train L1: 0.0062, train L2: 0.0738 
Iteration 31, val Loss: 0.0087, val L1: 0.0057, val L2: 0.0362 ||| train Loss: 0.0123 train L1: 0.0064, train L2: 0.0649 
Iteration 39, val Loss: 0.0087, val L1: 0.0057, val L2: 0.0361 ||| train Loss: 0.0136 train L1: 0.0065, train L2: 0.0776 
Iteration 47, val Loss: 0.0087, val L1: 0.0057, val L2: 0.0360 ||| train Loss: 0.0141 train L1: 0.0078, train L2: 0.0710 
Iteration 55, val Loss: 0.0087, val L1: 0.0057, val L2: 0.0360 ||| train Loss: 0.0123 train L1: 0.0055, train L2: 0.0736 
Iteration 63, val Loss: 0.0088, val L1: 0.0057, val L2: 0.0361 ||| train Loss: 0.0142 train L1: 0.0066, train L2: 0.0830 

{'step': 63,
 'train_loss': 0.014231725514533371,
 'train_l1_loss': 0.006592273684545699,
 'train_l2_loss': 0.08298679198442242,
 'val_loss': 0.008750413182970058,
 'val_l1_loss': 0.005709825562347014,
 'val_l2_loss': 0.03611570176857744,
 'epoch_time': 3.2728707790374756}

In [ ]:

visualizer = MosaicSDFVisualizer(optimizer.model, optimizer.shape_sampler, 
    device, template_mesh_path=cube_wireframe_path)#, requires_grad=False)

# visualizer.plot_meshes()

In [ ]:
def compare_shapes(resolution = 32, show_mosaic_grids = False):

    with torch.no_grad():

        gt_sdf_mesh = MosaicSDFVisualizer.rasterize_sdf(
            sdf_func=shape_sampler.forward, resolution=resolution, sdf_scaler=-1, 
            extra_sdf_offset=[2,0, 0], vert_colors=[0, .5, 0])

        gt_mesh = visualizer.create_state_meshes(
            show_mosaic_grids=False,
            show_target_mesh=True,
            show_boundary_mesh=False,
            resolution=resolution,
            show_rasterized_sdf_mesh=False,
            vert_colors=[0, 0, .5],
            offset_vertices=torch.tensor([-2,0,0], device=device)
            )
        
        meshes = visualizer.create_state_meshes(
            show_mosaic_grids=show_mosaic_grids,
            show_target_mesh=False,
            show_boundary_mesh=False,
            resolution=resolution,
            vert_colors=[.5, .5, 0]
            )
        
        # Render the plotly figure
        fig = plot_scene({
            "subplot1": {
                "mesh": meshes,
                'gt_sdf_mesh': gt_sdf_mesh,
                'gt_mesh': gt_mesh
            }
        })
        fig.show()    
    


In [ ]:
# compare_shapes(32, show_mosaic_grids=False)
compare_shapes(32, show_mosaic_grids=False)

In [ ]:

# Assuming you're not using a data loader for training
optimizer.train()  # Pass None or adjust `train` method to not require `train_loader`



# I am also not sure autograd is what we need, how it works in this case at all?

# # To save the optimized model
# optimizer.save_checkpoint('path/to/save/checkpoint')

# # To load an existing model
# optimizer.load_checkpoint('path/to/existing/checkpoint.pth')


In [ ]:
compare_shapes(32)

In [ ]:
# if True:
#     with torch.no_grad():
#         meshes = visualizer.create_state_meshes(
#             show_mosaic_grids=False,
#             show_target_mesh=False,
#             resolution=16
#             )
        
#         # Render the plotly figure
#         fig = plot_scene({
#             "subplot1": {
#                 "mesh": meshes
#             }
#         })
#         fig.show()    
    


In [ ]:
# with torch.no_grad():
#     resolution = 8
#     grid_points = torch.stack(torch.meshgrid(
#             torch.linspace(-1, 1, resolution),
#             torch.linspace(-1, 1, resolution),
#             torch.linspace(-1, 1, resolution)
#         ), dim=-1).reshape(-1, 3)#.to(device)

#     sdf_values = optimizer.model(grid_points.to(device))

In [ ]:
sdf_meshes._verts_list

In [ ]:
import torch
import numpy as np
from skimage.measure import marching_cubes
from pytorch3d.structures import Meshes
from pytorch3d.io import save_obj
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader
)
mosaic_sdf = optimizer.model
resolution = 16

# Assuming 'mosaic_sdf' is your MosaicSDF instance and 'resolution' is the desired grid resolution
grid_points = torch.stack(torch.meshgrid(
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution)
), dim=-1).reshape(-1, 3).to(config['device'])

# Get the SDF values at these points
sdf_values = mosaic_sdf(grid_points).detach().cpu().numpy()
sdf_volume = sdf_values.reshape(resolution, resolution, resolution)
sdf_volume.shape

In [ ]:
grid_points.shape

In [ ]:

# Run marching cubes to get vertices, faces, and normals
verts, faces, normals, _ = marching_cubes(sdf_volume, level=0)
faces = faces + 1  # skimage has 0-indexed faces, while PyTorch3D expects 1-indexed

# Convert to PyTorch tensors
verts = torch.tensor(verts, dtype=torch.float32)
faces = torch.tensor(faces, dtype=torch.int64)

# Create a PyTorch3D mesh
mesh = Meshes(verts=[verts], faces=[faces])

# Initialize a renderer
R, T = look_at_view_transform(2.7, 0, 90)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
)

# Render the mesh
images = renderer(mesh)


In [ ]:
num_check_x = torch.rand((3, 3))
forward = lambda x: torch.sum(x ** 2, axis=1)

num_grad = optimizer.compute_gradient_numerically(num_check_x, forward)

In [ ]:
num_grad

In [ ]:
num_check_x * 2

In [ ]:

# Define the quadratic function
def quadratic_function(points):
    return torch.sum(points ** 2, dim=-1)

# Known analytical gradients for the quadratic function
def analytical_gradients(points):
    return 2 * points

# Define points for which to compute gradients
points = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0]], requires_grad=True)

# Compute the function output
function_output = quadratic_function(points)

# Compute analytical gradients
true_gradients = analytical_gradients(points)

# Compute gradients using autograd
function_output.backward(torch.ones_like(function_output))
autograd_gradients = points.grad

# Compute gradients using the numerical method
numerical_gradients = optimizer.compute_gradient_numerically(points, quadratic_function, delta=1e-2)

# Compare numerical gradients to true gradients
gradient_difference = torch.abs(numerical_gradients - true_gradients)

print("True gradients:\n", true_gradients)
print("Autograd gradients:\n", autograd_gradients)
print("Numerical gradients:\n", numerical_gradients)
print("Gradient difference:\n", gradient_difference)

# Check if the numerical gradients are close to the true gradients
assert torch.allclose(numerical_gradients, true_gradients, atol=1e-3), "Numerical gradients do not match true gradients closely enough."


In [ ]:
## test vars

volume_centers = torch.tensor([
    [0.1, 0, 0.5], 
    [0.5, 0.2, 0], 
    [0.7, 0.1, 0.6], 
    [1, 1, .5], 
])

scales = torch.tensor([1, 1, 1, .4])

points = torch.tensor([
    [0.5, 0.5, 0], 
    [1, 0, .5], 
])

k = 7

sdf_values = torch.rand((scales.shape[0], k, k ,k))


In [ ]:
# # Using the updated ShapeSampler class
# shape_sampler = ShapeSampler("sphere")
# center = torch.tensor([0.0, 0.0, 0.0])
# scale = 2.0

# # Compute local SDF values
# local_sdf_values = compute_local_sdf(shape_sampler, center, scale, grid_resolution=3)
# print(local_sdf_values.shape) 